## Motor de recomendación, biblioteca Luis Ángel Arango
### Miguel García
### Mariana León

## Procesamiento de los datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
%%time
datos = pd.read_excel("CIRCULA_23_TRANSF_S.xlsx", sheet_name="CIRCULA_2023")

CPU times: total: 7min 51s
Wall time: 17min 31s


In [3]:
datos.head(4)

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Fecha de ingreso del Item,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,1997-12-30 00:00:00,Categoría Biblioteca Virtual,3900488,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,1997-02-24 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO
2,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004007799355,991008897989707486,...,2000-05-22 00:00:00,Categoría Biblioteca Virtual,3900488,986.251 B82,0,986.251 B82,986,986.0,980,LIBRO
3,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004017298190,991005656579707486,...,2005-03-22 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45v2,0,928.61 S45v2,928,928.0,920,LIBRO


### Usando los prestamos externos

In [4]:
datos.shape

(807075, 30)

In [5]:
datos1 = datos[datos['Prestamos Externo'] == 1]
datos1.shape

(362338, 30)

In [6]:
prestamos = datos1.get(['Identificador Socio', 'Título','Fecha de préstamo'])
prestamos.shape

(362338, 3)

### Registros duplicados

In [7]:
frecuencia = prestamos[['Identificador Socio','Título']].value_counts().reset_index()
duplicados = frecuencia[frecuencia['count'] > 1]
duplicados.head()

,Identificador Socio,Título,count
0,39004880069254,el pais,61
1,39004880023558,noticias historiales de las conquistas de tier...,55
2,39004880041808,el espectador,36
3,196366477,revista de medicina legal de colombia,34
4,197183031,cromos revista semanal ilustrada,30


In [8]:
round((duplicados.shape[0]/prestamos.shape[0])*100,2)

9.71

El 9.71% de los datos se quitan al ser repeticiones de un mismo prestamo.

In [9]:
duplicados.head()

,Identificador Socio,Título,count
0,39004880069254,el pais,61
1,39004880023558,noticias historiales de las conquistas de tier...,55
2,39004880041808,el espectador,36
3,196366477,revista de medicina legal de colombia,34
4,197183031,cromos revista semanal ilustrada,30


In [10]:
prestamos1 = prestamos.drop_duplicates(subset=['Identificador Socio','Título'])
frecuencia1 = prestamos1[['Identificador Socio','Título']].value_counts().reset_index()
frecuencia1.head()

,Identificador Socio,Título,count
0,000184724,acordeones cumbiamba y vallenato en el magdale...,1
1,39004885699246,ideas socialistas en colombia,1
2,39004885699246,escritura o la vida,1
3,39004885699246,condenados de la tierra,1
4,39004885699246,arte de amar,1


In [11]:
verificacion  = duplicados[['Identificador Socio','Título']].apply(tuple, axis=1).isin(frecuencia1[['Identificador Socio','Título']].apply(tuple, axis=1))
verificacion.unique()

array([ True])

Se verifico que no se eliminaron las observaciones con duplicados, sino solo sus copias.

### Libros que solo se han prestado una vez

In [12]:
frec_libro = prestamos1['Título'].value_counts().reset_index()
libros_pocos = frec_libro[frec_libro['count']==1]
libros_pocos.head()

,Título,count
39564,arquitectura caribena puerto limon bocas del toro,1
39565,contratacion electronica,1
39566,vulnerabilidad y derechos de la ninez,1
39567,seis noches en la acropolis,1
39568,fortalezas y debilidades del nuevo codigo de p...,1


In [13]:
round((libros_pocos.shape[0]/prestamos1.shape[0])*100,2)

18.35

Al rededor del 18.35% de las observaciones son de libros que se prestaron una unica vez

In [14]:
prestamos1.shape

(309708, 3)

In [15]:
prestamos2 = prestamos1[~prestamos1['Título'].isin(libros_pocos['Título'])]
prestamos2.shape

(252892, 3)

### Usuarios que han sacado dos libros

In [16]:
frec_socio = prestamos2['Identificador Socio'].value_counts().reset_index()
socios_pocos = frec_socio[frec_socio['count'] < 3]
socios_pocos.head()

,Identificador Socio,count
21762,39004886348058,2
21763,201332905,2
21764,201273517,2
21765,201433049,2
21766,200919904,2


In [17]:
round((socios_pocos.shape[0]/prestamos2.shape[0])*100,2)

4.67

El 4.67% de usuarios no son aptos para el motor de recomendación por poco uso

In [18]:
prestamos3 = prestamos2[~prestamos2['Identificador Socio'].isin(socios_pocos['Identificador Socio'])]
prestamos3.shape

(236096, 3)

### Codificación de usuarios y libros

In [19]:
prestamos3.head()

,Identificador Socio,Título,Fecha de préstamo
0,3900488,poesia y prosa con 44 textos sobre el autor,2023-05-12
1,3900488,jose asuncion silva el corazon del poeta,2023-03-15
2,3900488,bucaramanga pasado y presente,2023-02-22
3,3900488,almas en pena chapolas negras,2023-03-15
6,3900488,santander en mis mejores fotografias,2023-02-22


In [20]:
cod_socios = LabelEncoder()
cod_libros = LabelEncoder()

prestamos3.loc[:,'COD_SOCIO'] = cod_socios.fit_transform(prestamos3['Identificador Socio'])
prestamos3.loc[:,'COD_LIBRO'] = cod_libros.fit_transform(prestamos3['Título'])

C:\Users\acer\AppData\Local\Temp\ipykernel_12528\228931272.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos3.loc[:,'COD_SOCIO'] = cod_socios.fit_transform(prestamos3['Identificador Socio'])
C:\Users\acer\AppData\Local\Temp\ipykernel_12528\228931272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos3.loc[:,'COD_LIBRO'] = cod_libros.fit_transform(prestamos3['Título'])


In [21]:
prestamos3.head()

,Identificador Socio,Título,Fecha de préstamo,COD_SOCIO,COD_LIBRO
0,3900488,poesia y prosa con 44 textos sobre el autor,2023-05-12,14571,29753
1,3900488,jose asuncion silva el corazon del poeta,2023-03-15,14571,20392
2,3900488,bucaramanga pasado y presente,2023-02-22,14571,4532
3,3900488,almas en pena chapolas negras,2023-03-15,14571,1295
6,3900488,santander en mis mejores fotografias,2023-02-22,14571,32931


In [22]:
prestamos4 = prestamos3.get(['COD_SOCIO','COD_LIBRO','Fecha de préstamo'])
prestamos4.head()

,COD_SOCIO,COD_LIBRO,Fecha de préstamo
0,14571,29753,2023-05-12
1,14571,20392,2023-03-15
2,14571,4532,2023-02-22
3,14571,1295,2023-03-15
6,14571,32931,2023-02-22


## Desarrollo motor de recomendación

### Train y Test

In [23]:
prestamos4['Fecha de préstamo'] = pd.to_datetime(prestamos4['Fecha de préstamo'])
prestamos4.loc[:,'TIEMPO'] = prestamos4['Fecha de préstamo'].astype('int64') // 10**9
prestamos4.loc[:,'ORDEN'] = prestamos4.groupby(['COD_SOCIO'])['TIEMPO'].rank(method='first', ascending=False)

C:\Users\acer\AppData\Local\Temp\ipykernel_12528\1532861923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos4['Fecha de préstamo'] = pd.to_datetime(prestamos4['Fecha de préstamo'])
C:\Users\acer\AppData\Local\Temp\ipykernel_12528\1532861923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos4.loc[:,'TIEMPO'] = prestamos4['Fecha de préstamo'].astype('int64') // 10**9


In [24]:
train = prestamos4[prestamos4['ORDEN'] != 1]
test = prestamos4[prestamos4['ORDEN'] == 1]

train = train[['COD_SOCIO','COD_LIBRO']]
test = test[['COD_SOCIO','COD_LIBRO']]

In [25]:
verificacion  = test[['COD_SOCIO','COD_LIBRO']].apply(tuple, axis=1).isin(train[['COD_SOCIO','COD_LIBRO']].apply(tuple, axis=1))
verificacion.unique()

array([False])

Se verifica que los registros en la base de test no están en la base de train

### Entrenamiento Red Neuronal

In [26]:
class MovieLensTrainDataset(Dataset):
    def __init__(self, datos):
        self.socios, self.libros, self.labels = self.get_dataset(datos)

    def __len__(self):
        return len(self.socios)

    def __getitem__(self, idx):
        return self.socios[idx], self.libros[idx], self.labels[idx]

    def get_dataset(self, datos):
        socios, libros, labels = [], [], []
        user_item_set = set(zip(datos['COD_SOCIO'], datos['COD_LIBRO']))
        set_sl = datos['COD_LIBRO'].unique()
        
        num_negatives = 4
        for u, i in user_item_set:
            socios.append(u)
            libros.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(set_sl)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(set_sl)
                socios.append(u)
                libros.append(negative_item)
                labels.append(0)

        return torch.tensor(socios), torch.tensor(libros), torch.tensor(labels)

In [27]:
class NCF(pl.LightningModule):
    def __init__(self, num_users, num_items, datos):
        super().__init__()
        self.socio_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=128)
        self.libro_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=128)
        self.fc1 = nn.Linear(in_features=256, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=128)
        self.fc4 = nn.Linear(in_features=128, out_features=64)
        self.output = nn.Linear(in_features=64, out_features=1)
        self.datos = datos

    def forward(self, user_input, item_input):
        user_embedded = self.socio_embedding(user_input)
        item_embedded = self.libro_embedding(item_input)

        vector = torch.cat([user_embedded, item_embedded], dim=-1)
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))
        vector = nn.ReLU()(self.fc3(vector))
        vector = nn.ReLU()(self.fc4(vector))

        pred = nn.Sigmoid()(self.output(vector))
        return pred

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.1)

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.datos),
                          batch_size=512)

In [28]:
num_users = prestamos4['COD_SOCIO'].max()+1
num_items = prestamos4['COD_LIBRO'].max()+1
all_set = prestamos4['COD_LIBRO'].unique()

In [29]:
model = NCF(num_users, num_items, train)
trainer = pl.Trainer(max_epochs=5, accelerator='gpu', reload_dataloaders_every_n_epochs=1, enable_progress_bar=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [30]:
%%time
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type      | Params
----------------------------------------------
0 | socio_embedding | Embedding | 2.8 M 
1 | libro_embedding | Embedding | 5.0 M 
2 | fc1             | Linear    | 131 K 
3 | fc2             | Linear    | 131 K 
4 | fc3             | Linear    | 32.9 K
5 | fc4             | Linear    | 8.3 K 
6 | output          | Linear    | 65    
----------------------------------------------
8.1 M     Trainable params
0         Non-trainable params
8.1 M     Total params
32.467    Total estimated model params size (MB)
C:\Users\acer\anaconda3\envs\my_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: total: 3min 56s
Wall time: 11min 43s


### Evaluación del motor de recomendación

In [31]:
test_set = set(zip(test['COD_SOCIO'], test['COD_LIBRO']))
interacciones = prestamos4.groupby('COD_SOCIO')['COD_LIBRO'].apply(list).to_dict()

In [32]:
%%time
hits = []
predicciones = []
for (u,i) in test_set:
    interacted_items = interacciones[u]
    not_interacted_items = set(all_set) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]

    predicted_labels = np.squeeze(model(torch.tensor([u]*100),
                                        torch.tensor(test_items)).detach().numpy())

    top20_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:20].tolist()]

    if i in top20_items:
        hits.append(1)
        predicciones.append([u,i])
    else:
        hits.append(0)

print("The Hit Ratio @ 20 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 20 is 1.00
CPU times: total: 5min 23s
Wall time: 9min 54s


#### Se tiene un HIT RATIO 20 de 1